In [1]:
using IntervalArithmetic
using LaTeXStrings
using Plots

In [3]:
include("./method1/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")

print()

In [8]:
function laTeXMatrix(
        pattern,
        size::Tuple{Integer, Integer})::String
    m, n = size

    mat = "\\begin{bmatrix}"
    for i = 1:m, j = 1:n
        mat *= pattern(i, j)
        if j == n
            if i != m
                mat *= " \\\\ "
            end
        else
            mat *= " & "
        end
    end
    mat *= "\\end{bmatrix}"

    return mat
end

laTeXMatrix (generic function with 1 method)

In [14]:
function method1(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    _, n = size(A₁)

    A = intervalPCM2TwofoldIntervalPCM(A₁, A₂)
    println("A = $(twofoldIntervalMatrixLaTeXString(A))"); println()

    result = solveLP_m1(A)
    resultₛₜᵣ = lPResultLaTeXString_m1(result)
    println("$(laTeXMatrix((i,j) -> "\\left[ w_$i^\\text{L}, w_$i^\\text{U} \\right]", (n, 1))) = $(resultₛₜᵣ.W)"); println()
    println("""
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{L}-}", (n, 1))) = $(resultₛₜᵣ.wᴸ⁻), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{U}-}", (n, 1))) = $(resultₛₜᵣ.wᵁ⁻), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{L}+}", (n, 1))) = $(resultₛₜᵣ.wᴸ⁺), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{U}+}", (n, 1))) = $(resultₛₜᵣ.wᵁ⁺)
        """); println()
    println("""
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{L}", (n, 1))) = $(resultₛₜᵣ.εᴸ), ~~
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{U}", (n, 1))) = $(resultₛₜᵣ.εᵁ)
        """); println()

    Â = updatePCM_m1(A, result)

    return
end

method1 (generic function with 1 method)

# example 1

In [15]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
A₂ = A₁

method1(A₁, A₂)

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   2.0 \right],   2.0 \right] & \left[ 0.667, \left[ 0.667, 1.333 \right], 1.333 \right] & \left[   1.0, \left[   1.0, 1.333 \right], 1.333 \right] \\ \left[   0.5, \left[   0.5,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.444, \left[ 0.444,   1.0 \right],   1.0 \right] & \left[ 0.667, \left[ 0.667,   1.0 \right],   1.0 \right] \\ \left[  0.75, \left[  0.75,   1.5 \right],   1.5 \right] & \left[   1.0, \left[   1.0,  2.25 \right],  2.25 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   1.5 \right],   1.5 \right] \\ \left[  0.75, \left[  0.75,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   1.5 \right],   1.5 \right] & \left[ 0.667, \left[ 0.667,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

\begin{bmatrix}\left

# example 2

In [16]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4 / 5)..(A₁[i,j].hi * 5 / 4)
end

method1(A₁, A₂)

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.8, \left[   1.0,   2.0 \right],   2.5 \right] & \left[ 0.533, \left[ 0.667, 1.333 \right], 1.667 \right] & \left[   0.8, \left[   1.0, 1.333 \right], 1.667 \right] \\ \left[   0.4, \left[   0.5,   1.0 \right],  1.25 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.356, \left[ 0.444,   1.0 \right],  1.25 \right] & \left[ 0.533, \left[ 0.667,   1.0 \right],  1.25 \right] \\ \left[   0.6, \left[  0.75,   1.5 \right], 1.875 \right] & \left[   0.8, \left[   1.0,  2.25 \right], 2.812 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.8, \left[   1.0,   1.5 \right], 1.875 \right] \\ \left[   0.6, \left[  0.75,   1.0 \right],  1.25 \right] & \left[   0.8, \left[   1.0,   1.5 \right], 1.875 \right] & \left[ 0.533, \left[ 0.667,   1.0 \right],  1.25 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

\begin{bmatrix}\left

<!-- ともに弱い整合性を持ち， $A_{1ij} \subseteq A_{2ij}$

```
A₁ = [
    1..1 2..5 2..4 2..3
    (1/5)..(1/2) 1..1 (1/5)..4 (1/5)..3
    (1/4)..(1/2) (1/4)..5 1..1 (1/4)..3
    (1/3)..(1/2) (1/3)..5 (1/3)..4 1..1
]
A₂ = fill(1..1, size(A₁))
_, n = size(A₁)
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 1/1.1)..(A₁[i,j].hi * 1.1)
end

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

display(isConsistentIntervalPCM(A₁))
display(isWeaklyConsistentIntervalPCM(A₁))
display(isConsistentIntervalPCM(A₂))
display(isWeaklyConsistentIntervalPCM(A₂))

plotAll(A₁, A₂)
``` -->

# example 3

In [17]:
A = [
    1 2 4 4
    1/2 1 2 2
    1/4 1/2 1 1
    1/4 1/2 1 1
]

A₁ = randamizedIntervalPCM(A, 1234, 3e-2)
A₂ = randamizedIntervalPCM(A, 134, 3e-2)

method1(A₁, A₂)

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.981, \left[ 1.998,  2.01 \right], 2.033 \right] & \left[ 3.974, \left[ 3.974,  4.05 \right], 4.109 \right] & \left[ 3.958, \left[ 3.962, 4.041 \right], 4.048 \right] \\ \left[ 0.492, \left[ 0.497,   0.5 \right], 0.505 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.944, \left[ 1.945, 2.015 \right], 2.048 \right] & \left[ 1.942, \left[ 1.971, 2.033 \right], 2.045 \right] \\ \left[ 0.243, \left[ 0.247, 0.252 \right], 0.252 \right] & \left[ 0.488, \left[ 0.496, 0.514 \right], 0.515 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.972, \left[ 0.983, 1.021 \right], 1.022 \right] \\ \left[ 0.247, \left[ 0.247, 0.252 \right], 0.253 \right] & \left[ 0.489, \left[ 0.492, 0.507 \right], 0.515 \right] & \left[ 0.978, \left[ 0.979, 1.017 \right], 1.029 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

\begin{bmatrix}\left

# example 4

In [18]:
A = [
    1 2 3 10
    1/2 1 3 8
    1/3 1/3 1 3
    1/10 1/8 1/3 1
]

A₁ = randamizedIntervalPCM(A, 124, 3.5e-1)
A₂ = randamizedIntervalPCM(A, 4, 3.5e-1)

method1(A₁, A₂)

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.477, \left[ 1.483, 2.432 \right], 2.772 \right] & \left[ 2.334, \left[ 2.367, 3.578 \right], 3.893 \right] & \left[ 8.353, \left[ 8.663, 11.158 \right], 12.974 \right] \\ \left[ 0.361, \left[ 0.411, 0.674 \right], 0.677 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 2.216, \left[ 2.786, 3.049 \right], 3.835 \right] & \left[ 6.615, \left[ 6.654, 8.351 \right], 10.334 \right] \\ \left[ 0.257, \left[ 0.279, 0.422 \right], 0.428 \right] & \left[ 0.261, \left[ 0.328, 0.359 \right], 0.451 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 2.263, \left[ 2.648, 3.309 \right], 4.048 \right] \\ \left[ 0.077, \left[  0.09, 0.115 \right],  0.12 \right] & \left[ 0.097, \left[  0.12,  0.15 \right], 0.151 \right] & \left[ 0.247, \left[ 0.302, 0.378 \right], 0.442 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

\begin{bmatrix}\l